In [ ]:
# !pip install OpenDartReader

In [2]:
from financial_data_making import financial_dict
import OpenDartReader
import pandas as pd
import time 

# df = pd.read_csv('./merged_code.csv')
call =  financial_dict('1ee244d6168bde153cb2d45463d71139f567d1f4') # api_key

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': 'str', 'stock_code': 'str'})

df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
df['year'] = df['year'].astype('int')
print(df.dtypes)
print(df.shape)
# df.to_csv('final_corp_info.csv')
df.head(3)

corp_name     object
esg           object
fin_state     object
corp_code     object
stock_code    object
year           int32
dtype: object
(2541, 6)


,corp_name,esg,fin_state,corp_code,stock_code,year
0,기아,A+,"60,490,443,000,000",00106641,000270,2020
1,S-Oil,A+,"15,690,510,000,000",00138279,010950,2020
2,풀무원,A+,"1,693,573,176,270",00155355,017810,2020


In [ ]:
api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)
code, year = '311690', 2020

# 매출액
# 손익계산서
def sales_docu(code, year):
    sales = dart.finstate(code, year)
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액 
    c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values[0] # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values[0]

    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%)
    ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3

    exec = dart.report(code, '임원', year) #'사외이사' / 전체 임원 수   
    out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    exec = exec.shape[0]

    employee = dart.report(code, '직원', year)['rgllbr_co'].to_list() # 총 직원 수
    
    output = {'sales': c_sales, 
              'pre_sales': p_sales, 
              'cash_div': c_div, 
              'pre_cash_civ': p_div,
              'stakeholder' : ratio_stakeholder,
              'num_outexecutives': out_exec,
              'num_executives': exec,
              'num_employee': employee}
    
    return output

In [ ]:
# 데이터 수집 (1)

data = []
good_index = []
bad_index = []

for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data.append(sales_docu(code, year))
        good_index.append(index)
    except:
        bad_index.append(index)
print(len(good_index), len(bad_index)) # 

# data downloading
with open ('s_docu.txt', 'w', encoding='utf-8') as f:
    for x in df.iloc[good_index, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_docu.txt', 'w', encoding='utf-8') as f:
    for x in df.iloc[bad_index, 0].values:
        f.write(f'{x}\n')
    f.close()

In [ ]:
api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

def report_docu(code, year):
    lst = dart.list(code, start='2020', end='2023') # 2020-2023 자료 중에 데이터 찾기
    
    # 변경1: 모든 사업 보고서에서 원하는 년도의 사업 보고서 고르기 
    report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(year)))].iloc[0, 5]
    url = dart.sub_docs(report_num) #사업보고서
    html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1]
    
    docu = pd.read_html(html)
    if docu[1].empty | docu[3].empty:
        print('Some document cannot be extracted')
    else:
        fin_state = docu[1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

        asset = fin_state[fin_state['tag'] == '자산총계'] # 원래는 iloc[0, 1] # 자산 총계
        debt= fin_state[fin_state['tag'] == '부채총계'] # 원래는 iloc[0, 1] # 부채 총계
        intangible = fin_state[fin_state['tag'] == '무형자산'] # 원래는 iloc[0, 1] # 무형자산 

        income = docu[3].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서

        profit = income[income['tag'].str.contains('영업이익')] # 원래는 iloc[0, 1] # ()는 마이너스를 뜻함 # 영업이익(손실)
        # profit_tax = income[income['tag'].str.contains('법인세비용차감전')] # 원래는 iloc[0, 1] # t기 법인세비용차감전순이익(손실)
        
        df = pd.concat([asset, debt, intangible, profit], axis=0).reset_index(drop=True)
        
        # dict_2 = {'asset': asset,
        #             'debt': debt,
        #             'intang': intangible,
        #             'profit': profit,
        #             'profit_without_tax': c_profit_tax,
        #             'pre_profit_without_tax': p_profit_tax}   
    return df                                                                                            

In [ ]:
print(report_docu('034730', 2020))

In [25]:
api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

lst = dart.list('034730', start='2020', end='2023') # 2020-2023 자료 중에 데이터 찾기
    
# 변경1: 모든 사업 보고서에서 원하는 년도의 사업 보고서 고르기 
report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(2020)))].iloc[0, 5]
url = dart.sub_docs(report_num) #사업보고서
html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1]

docu = pd.read_html(html)
if docu[1].empty | docu[3].empty:
    print('Some document cannot be extracted')
else:
    fin_state = docu[1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

    asset = fin_state[fin_state['tag'] == '자산총계'] # 원래는 iloc[0, 1] # 자산 총계
    debt= fin_state[fin_state['tag'] == '부채총계'] # 원래는 iloc[0, 1] # 부채 총계
    intangible = fin_state[fin_state['tag'] == '무형자산'] # 원래는 iloc[0, 1] # 무형자산 

    income = docu[3].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서

    profit = income[income['tag'].str.contains('영업이익')] # 원래는 iloc[0, 1] # ()는 마이너스를 뜻함 # 영업이익(손실)
    # profit_tax = income[income['tag'].str.contains('법인세비용차감전')] # 원래는 iloc[0, 1] # t기 법인세비용차감전순이익(손실)
    pd.concat([asset, debt, intangible, profit], axis=0).reset_index(drop=True)
    
profit

,tag,제 30 기,제 29 기,제 28 기
4,영업이익(손실),"(164,471)",3735874,4416501
10,법인세비용차감전계속영업이익(손실),"(27,806)",2835803,7295314
31,보통주 기본 및 희석주당계속영업이익 (단위 : 원),2146,14384,32375


In [30]:
pd.concat([asset, debt, intangible, profit], axis=0).reset_index(drop=True)

,tag,제 30 기,제 29 기,제 28 기
0,자산총계,137638377,132611423.0,119456991
1,부채총계,85778459,80435391.0,68569119
2,무형자산,11904440,12067786.0,13811199
3,영업이익(손실),"(164,471)",3735874,4416501
4,법인세비용차감전계속영업이익(손실),"(27,806)",2835803,7295314
5,보통주 기본 및 희석주당계속영업이익 (단위 : 원),2146,14384,32375


In [ ]:
lst = dart.list('034730', start='2020', end='2023')
report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(2020)))].iloc[0, 5]#'rcept_no']
report_num

In [ ]:
# 데이터 수집 (2)

data_re = []
good_index_re = []
bad_index_re = []

for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data_re.append(report_docu(code, str(year)))
        print(f'{index}, Good!')
        good_index_re.append(index)
    except:
        print(f'{index}, bad...')
        bad_index_re.append(index)
        
print(len(good_index_re), len(bad_index_re)) # 


In [ ]:
succees_report_docu = df.iloc[good_index_re, 0]
fail_report_docu = df.iloc[bad_index_re, 0]

# data downloading
with open ('s_report.txt', 'w') as f:
    for x in df.iloc[good_index_re, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_report.txt', 'w') as f:
    for x in df.iloc[bad_index_re, 0].values:
        f.write(f'{x}\n')
    f.close()
    
import pickle
with open('data_report.pickle', 'wb') as f:
    # for  data in data_re
    pickle.dump(data_re, f)

In [ ]:
with open('data_report.pickle', 'rb') as f:
    loaded = pickle.load(f)
print(loaded)
print(len(loaded))